In [26]:
import pandas as pd
import json
import folium

In [3]:
df = pd.read_parquet("../data/data_counts.parquet")
df.keys()

Index(['file', 'BUILDING', 'NPIP', 'COUNTRY', 'POPULATED_PLACE', 'DLF',
       'SPATIAL_OBJ', 'REGION', 'ENV_FEATURES', 'INT_SPACE', 'RIVER', 'FOREST',
       'BUILDING_texts', 'NPIP_texts', 'COUNTRY_texts',
       'POPULATED_PLACE_texts', 'DLF_texts', 'SPATIAL_OBJ_texts',
       'REGION_texts', 'ENV_FEATURES_texts', 'INT_SPACE_texts', 'RIVER_texts',
       'FOREST_texts', 'RG Number', 'PDF URL', 'USHMM URL', 'First Name',
       'Middle Name', 'Last Name', 'Birth Name', 'Gender', 'Birth Date',
       'Birth Year', 'Place of Birth', 'Country', 'Experience Group',
       'Ghetto(s) Encyclopedia', 'Ghetto', 'Camp(s) Encyclopedia', 'Camp',
       'Non-SS Camp  ', 'Region', 'Needs Research', 'Data Entry', 'Accession',
       'Notes:', 'Revisit'],
      dtype='object')

In [30]:
df_map = pd.read_json("../data/coords_map.json")
places = df_map.keys().tolist()

place_dict = df_map.to_dict()
place_dict

{'Ahrensbök-Holstendorf': {0: 54.026144, 1: 10.593556},
 'Alt Daber': {0: 53.204755, 1: 12.501389},
 'Altenberg': {0: 50.766468, 1: 13.728242},
 'Ankenbuck': {0: 48.000711, 1: 8.5121907},
 'Anrath bei Krefeld': {0: 51.285816, 1: 6.455635},
 'Bad Sulza': {0: 51.089, 1: 11.625},
 'Bamberg': {0: 49.891919, 1: 10.896558},
 'Bautzen Kupferhammer': {0: 51.1901205, 1: 14.429039},
 'Bayreuth St. Georgen': {0: 49.9444809, 1: 11.571465},
 'Benninghausen': {0: 51.654883, 1: 8.244301},
 'Bergisch Gladbach': {0: 51.1, 1: 7.117},
 'Stellawerk': {0: 50.983299, 1: 7.133299},
 'Bergkamen-Schönhausen': {0: 51.613306, 1: 7.632339},
 'Berlin General-Pape-Strasse': {0: 52.475523, 1: 13.370051},
 'Berlin-Charlottenburg Maikowski-Haus': {0: 52.515886, 1: 13.314255},
 'Berlin-Köpenick': {0: 52.448123, 1: 13.569699},
 'Berlin-Kreuzberg Friedrichstrasse Nr. 234': {0: 52.502504, 1: 13.391008},
 'Berlin-Kreuzberg Hedemannstrasse': {0: 52.502588, 1: 13.386896},
 'Berlin-Plötzensee': {0: 52.540704, 1: 13.322347},
 

In [8]:
len(places)

4579

In [12]:
hits = []
for idx, row in df.iterrows():
    for place in row["POPULATED_PLACE_texts"]:
        if place in places:
            hits.append(place)


In [13]:
len(hits)

18377

In [14]:
from collections import Counter

In [18]:
counter = Counter(hits)

In [24]:
df2 = pd.DataFrame(list(counter.items()), columns=['Place', 'Count'])
df2

,Place,Count
0,Munich,778
1,Smela,1
2,Starodub,1
3,Orsha,25
4,Berlin,2498
...,...,...
335,Barth,2
336,Schlieben,2
337,Tence,1
338,Vittel,2


In [43]:
# Adding latitude and longitude columns
df2['Latitude'] = df2['Place'].apply(lambda x: place_dict[x][0] if x in place_dict else None)
df2['Longitude'] = df2['Place'].apply(lambda x: place_dict[x][1] if x in place_dict else None)


# Create a map
m = folium.Map(location=[50, 20], zoom_start=5) # Set appropriate starting location

def normalize_count(count):
    # Example normalization, adjust as needed
    return count * 10

# Add circle markers to the map
for idx, row in df2.iterrows():
    if pd.notnull(row['Latitude']) and pd.notnull(row['Longitude']):
        folium.Circle(
            location=[row['Latitude'], row['Longitude']],
            radius=normalize_count(row['Count']),  # Circle size based on count
            color='blue',
            fill=True,
            fill_color='blue',
            popup=row['Place']
        ).add_to(m)

# Save map to an HTML file
m.save('map.html')

In [28]:
places

['Ahrensbök-Holstendorf',
 'Alt Daber',
 'Altenberg',
 'Ankenbuck',
 'Anrath bei Krefeld',
 'Bad Sulza',
 'Bamberg',
 'Bautzen Kupferhammer',
 'Bayreuth St. Georgen',
 'Benninghausen',
 'Bergisch Gladbach',
 'Stellawerk',
 'Bergkamen-Schönhausen',
 'Berlin General-Pape-Strasse',
 'Berlin-Charlottenburg Maikowski-Haus',
 'Berlin-Köpenick',
 'Berlin-Kreuzberg Friedrichstrasse Nr. 234',
 'Berlin-Kreuzberg Hedemannstrasse',
 'Berlin-Plötzensee',
 'Berlin-Prenzlauer Berg',
 'Wasserturm',
 'Berlin-Spandau',
 'Berlin-Tiergarten Universum-Landesausstellungspark',
 'Bochum',
 'Börgermoor',
 'Papenburg I',
 'Börnicke',
 'Meissnershof',
 'Brandenburg an der Havel',
 'Brauweiler',
 'Breitenau',
 'Breslau-Dürrgoy',
 'Colditz',
 'Columbia-Haus',
 'Dresden Mathildenstrasse',
 'Düsseldorf Ulmenstrasse',
 'Ulmer Höh',
 'Erfurt Petersburg and Feldstrasse',
 'Esterwegen, IKL',
 'Esterwegen II',
 'Papenburg II',
 'Esterwegen III',
 'Papenburg III',
 'Eutin',
 'Fuhlsbüttel',
 'Hamburg-Fuhlsbüttel',
 'Glück